# 인트로

지난 튜토리얼에서 우리는 데이터프레임 또는 시리즈에서 데이터를 선택하는 방법을 배웠습니다. 연습 문제에서 설명한 것처럼 올바른 데이터를 추출하는 것은 작업을 완료하는 데 매우 중요합니다.

그러나 데이터가 항상 우리가 원하는 형식으로 나타나지는 않습니다. 때로는 작업에 맞게 데이터를 다시 포맷하기 위해 몇 가지 작업을 추가로 해야 합니다. 본 튜토리얼에서는 "적절한" 입력을 얻기 위해 적용될 수 있는 다양한 연산에 대해 설명합니다.

설명을 위해 와인 매거진 데이터를 이용할 것입니다.

In [1]:
import pandas as pd
pd.set_option('max_rows', 5)
import numpy as np
reviews = pd.read_csv("../../data/wine-reviews/winemag-data-130k-v2.csv", index_col=0)

In [2]:
reviews

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss
129970,France,"Big, rich and off-dry, this is powered by inte...",Lieu-dit Harth Cuvée Caroline,90,21.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Schoffit 2012 Lieu-dit Harth Cuvée Car...,Gewürztraminer,Domaine Schoffit


# 요약 함수

판다스는 데이터를 유용한 방식으로 재구성하는 많은 간단한 "요약 함수"를 (정식 명칭은 아닙니다) 제공합니다. 예를 들어 `descibe()` 메소드를 보면:

In [3]:
reviews.points.describe()

count    129971.000000
mean         88.447138
             ...      
75%          91.000000
max         100.000000
Name: points, Length: 8, dtype: float64

이 메소드는 주어진 열의 어트리뷰트에 대한 높은 수준의 요약을 생성합니다. 이는 입력의 데이터 타입에 따라 출력이 다릅니다. 위 출력은 오직 숫자 데이터에만 유효합니다; 문자열 데이터에 대해서는 다음을 얻을 수 있습니다:

In [4]:
reviews.taster_name.describe()

count         103727
unique            19
top       Roger Voss
freq           25514
Name: taster_name, dtype: object

데이터프레임의 열이나 시리즈의 특정 요약 통계량을 얻고 싶은 경우, 이를 가능하게 하는 유용한 판다스 함수가 대개 존재합니다.

예를 들어, 할당된 점수의 평균을 (예: 와인의 평균 등급) 알고 싶다면 `mean()` 함수를 사용하면 됩니다:

In [5]:
reviews.points.mean()

88.44713820775404

고유한 값의 목록을 보려면 `unique()` 함수를 사용합니다:

In [6]:
reviews.taster_name.unique()

array(['Kerin O’Keefe', 'Roger Voss', 'Paul Gregutt',
       'Alexander Peartree', 'Michael Schachner', 'Anna Lee C. Iijima',
       'Virginie Boone', 'Matt Kettmann', nan, 'Sean P. Sullivan',
       'Jim Gordon', 'Joe Czerwinski', 'Anne Krebiehl\xa0MW',
       'Lauren Buzzeo', 'Mike DeSimone', 'Jeff Jenssen',
       'Susan Kostrzewa', 'Carrie Dykes', 'Fiona Adams',
       'Christina Pickard'], dtype=object)

고유한 값의 목록과 그들이 데이터셋에 얼마나 자주 등장하는지 보기 위해서는 `value_counts()` 메소드를 사용하면 됩니다:

In [7]:
reviews.taster_name.value_counts()

Roger Voss           25514
Michael Schachner    15134
                     ...  
Fiona Adams             27
Christina Pickard        6
Name: taster_name, Length: 19, dtype: int64

# 맵

**맵(map)** 은 수학에서 차용한 용어로, 하나의 집합을 다른 집합에 "매핑"하는 함수를 나타내는 용어입니다. 데이터 과학에서는 종종 기존 데이터로 새로운 표현을 만들거나 현재 형식에서 원하는 형식으로 데이터를 변환해야 합니다. 맵은 이러한 작업을 처리하는 데 매우 중요합니다.

자주 사용되는 매핑 메소드 두 가지가 있습니다.

[`map()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html#pandas-series-map)이 그중 첫째로, 다른 하나보다 약간 간단합니다. 예를 들어, 와인의 평균 점수를 0으로 환산하려 한다고 가정합시다. 이 작업은 다음과 같이 수행할 수 있습니다:

In [8]:
review_points_mean = reviews.points.mean()
reviews.points.map(lambda p: p - review_points_mean)

0        -1.447138
1        -1.447138
            ...   
129969    1.552862
129970    1.552862
Name: points, Length: 129971, dtype: float64

`map()`에 전달하는 함수는 시리즈에서 단일 값(위 예시에서 point 값)을 받고, 그 값의 변환된 버전을 반환해야 합니다. `map()`은 함수에 의해 모든 값이 변환된 새로운 시리즈를 반환합니다.

[`apply()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html#pandas-dataframe-apply)는 각 행에 대해 사용자 지정 메소드를 호출하여 전체 데이터프레임을 변환할 때 사용하는 동일 메소드입니다.

In [9]:
def remean_points(row):
    row.points = row.points - review_points_mean
    return row

reviews.apply(remean_points, axis='columns')

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,-1.447138,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,-1.447138,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,1.552862,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss
129970,France,"Big, rich and off-dry, this is powered by inte...",Lieu-dit Harth Cuvée Caroline,1.552862,21.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Schoffit 2012 Lieu-dit Harth Cuvée Car...,Gewürztraminer,Domaine Schoffit


`axis='index'`를 포함하여 `reviews.apply()`를 호출하면, 행을 변환하는 함수가 아닌, *열* 을 변환하는 함수를 전달해야 합니다.

`map()`과 `apply()`는 각각 새로운 시리즈와 데이터프레임을 반환함을 주의해야 합니다. 이들은 호출된 원본 데이터를 수정하지 않습니다. `reviews`의 첫 행을 보면 본래의 `points` 값을 가짐을 확인할 수 있습니다.

In [10]:
reviews.head(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia


판다스는 일반적인 매핑 연산자를 기본으로 제공합니다. 예를 들어, 다음은 points 열의 평균을 바꾸는 더 빠른 방법입니다:

In [11]:
review_points_mean = reviews.points.mean()
reviews.points - review_points_mean

0        -1.447138
1        -1.447138
            ...   
129969    1.552862
129970    1.552862
Name: points, Length: 129971, dtype: float64

이 코드에서는 왼쪽의 많은 값(시리즈의 모든 값)과 오른쪽의 단일 값(평균값) 사이에서 작업을 수행하였습니다. 판다스는 이 표현을 보고 데이터셋의 모든 값에서 평균값을 빼야 함을 알아냅니다.

판다스는 또한 같은 길이의 시리즈 사이에서 이러한 작업을 수행할 경우 무엇을 해야 하는지 이해합니다. 에를 들어, 데이터셋의 국가 및 지역 정보를 쉽게 결합하는 방법은 다음과 같습니다:

In [12]:
reviews.country + " - " + reviews.region_1

0            Italy - Etna
1                     NaN
               ...       
129969    France - Alsace
129970    France - Alsace
Length: 129971, dtype: object

이러한 연산자들은 판다스에 내장된 속도 향상 기능을 사용하기에 `map()`이나 `apply()`보다 빠릅니다. 모든 표준 파이썬 연산자(`>`, `<`, `==` 등)는 이러한 방식으로 작동합니다.

그러나 덧셈과 뺄셈만으로는 할 수 없는 조건부 논리를 적용하는 것과 같이 고급스러운 일을 할 수 있는 `map()`이나 `apply()`처럼 유연하지 않습니다.